# 🐼 Pandas CheatSheet: Análisis de Datos en Python

## 1. Carga de Datos (Ingestión)

| Función | Descripción |
| --- | --- |
| **`pd.read_csv()`** | Carga archivos CSV (el más común). |
| **`pd.read_excel()`** | Carga archivos de Excel. |
| **`pd.read_sql()`** | Ejecuta consulta SQL y trae datos a un DataFrame. |
| **`pd.read_parquet()`** | Lectura de archivos Parquet (estándar en Big Data). |

## 2. Exploración Inicial (Inspección)

* **`df.head(n)` / `df.tail(n)`**: Primeras o últimas  filas.
* **`df.info()`**: Resumen de tipos de datos, memoria y valores nulos.
* **`df.describe()`**: Estadísticas descriptivas (media, desviación, cuartiles).
* **`df.shape`**: Dimensión del DataFrame `(filas, columnas)`.
* **`df.value_counts()`**: Frecuencia de valores únicos en una columna.

## 3. Selección y Filtrado

* **`df.loc[]`**: Filtro por **etiquetas** (nombres de filas/columnas).
* **`df.iloc[]`**: Filtro por **posiciones** (índices numéricos).
* **`df.query()`**: Filtrado con sintaxis estilo SQL.
* **`df.unique()` / `df.nunique()`**: Valores únicos o su conteo.

### 3.1 Reducción Lógica (Validación)
Se usan para resumir resultados de filtros booleanos.
* **`df.any()`**: Devuelve `True` si **al menos un** elemento es True (¿Existe algún error/nulo?).
* **`df.all()`**: Devuelve `True` si **todos** los elementos son True (¿Cumplen todos la regla?).
* **Uso con `axis=1`**: Para evaluar condiciones a lo largo de una fila completa.

## 4. Limpieza de Datos (Data Cleaning)

| Función | Acción |
| --- | --- |
| **`df.dropna()`** | Elimina filas con valores nulos (`NaN`). |
| **`df.fillna(v)`** | Remplaza nulos por un valor `v` (o la media). |
| **`df.drop_duplicates()`** | Elimina registros repetidos. |
| **`df.rename()`** | Renombra columnas o índices. |
| **`df.astype()`** | Cambia el tipo de dato (ej. de `object` a `int`). |

## 5. Manipulación y Agregación

* **`df.groupby()`**: Divide los datos para aplicar funciones por grupos.
* **`df.agg()`**: Aplica múltiples funciones estadísticas simultáneamente.
* **`df.pivot_table()`**: Crea tablas dinámicas con índices y columnas.
* **`df.sort_values()`**: Ordena el DataFrame por una o más columnas.
* **`df.apply()`**: Aplica una función (normal o `lambda`) a lo largo de un eje.

## 6. Combinación de Datasets

* **`pd.merge()`**: JOINs de bases de datos (Inner, Left, Right, Outer).
* **`pd.concat()`**: Unión física (pegar) de DataFrames por filas o columnas.
* **`df.join()`**: Unión optimizada basada en índices.

## 7. Series Temporales (Time Series)

* **`pd.to_datetime()`**: Convierte strings o números a objetos `datetime`.
* **`df.resample()`**: Agregación por frecuencias temporales (Día, Mes, Año).
* **`df.shift()`**: Desplaza los datos (útil para calcular variaciones porcentuales).

---

## 🚀 Ejemplo Práctico: Análisis Express de Ventas

Imagina que tenemos un dataset de ventas con nulos y queremos saber cuánto vendió cada categoría, pero solo de los productos caros.

In [1]:
import pandas as pd
import numpy as np

# 1. Crear datos de ejemplo
data = {
    'Fecha': ['2025-01-01', '2025-01-01', '2025-01-02', '2025-01-02'],
    'Categoria': ['Tech', 'Tech', 'Hogar', 'Tech'],
    'Precio': [1200, np.nan, 300, 1500],
    'Venta_ID': [1, 1, 2, 3]
}
df = pd.DataFrame(data)

# 2. Limpieza: Eliminar duplicados y llenar nulos con la media
df = df.drop_duplicates(subset='Venta_ID')
df['Precio'] = df['Precio'].fillna(df['Precio'].mean())

# 3. Transformación: Convertir a fecha y filtrar
df['Fecha'] = pd.to_datetime(df['Fecha'])
productos_premium = df.query("Precio > 500")

# 4. Agregación: Venta total por categoría
resultado = productos_premium.groupby('Categoria').agg({
    'Precio': ['sum', 'count', 'mean']
})

print(resultado)

           Precio              
              sum count    mean
Categoria                      
Tech       2700.0     2  1350.0


## Joins Complejos con `pd.merge()`

En proyectos reales, los datos no están en una sola tabla. El método `merge()` es el equivalente al `JOIN` de SQL y es extremadamente flexible.

### Tipos de Join (`how`)

* **Inner:** Solo registros con coincidencia en ambas tablas.
* **Left:** Todo lo de la izquierda + coincidencias de la derecha (el más usado).
* **Right:** Todo lo de la derecha + coincidencias de la izquierda.
* **Outer:** Todo lo de ambas tablas (rellena con `NaN` donde no hay cruce).

### Caso avanzado: Columnas con nombres distintos y sufijos

A veces, las columnas de unión no se llaman igual o hay columnas repetidas en ambos DataFrames.

```python
# Unir ventas con catálogo de productos
df_final = pd.merge(
    df_ventas, 
    df_productos, 
    left_on='id_prod_ventas',  # Nombre en la tabla A
    right_on='sku_producto',   # Nombre en la tabla B
    how='left',
    suffixes=('_venta', '_catalogo') # Evita colisiones de nombres
)

```

---

## Dominando las Series Temporales

Pandas nació en el sector financiero, por lo que su manejo de fechas es de nivel élite. El secreto está en convertir la columna de fecha en el **Índice (Index)** del DataFrame.

### El flujo de trabajo profesional:

1. **Convertir:** `pd.to_datetime()`
2. **Indexar:** `df.set_index('Fecha', inplace=True)`
3. **Operar:** Resampling o Rolling.

### Resampling vs. Rolling

* **`resample()`**: Cambia la frecuencia (ej. de datos diarios a la suma mensual).
* **`rolling()`**: Cálculos de ventana deslizante (ej. promedio móvil de los últimos 7 días).

### Ejemplo técnico:

```python
# Convertir y ordenar (vital para series temporales)
df['fecha'] = pd.to_datetime(df['fecha'])
df = df.set_index('fecha').sort_index()

# 1. Resample: Ventas totales por mes
ventas_mensuales = df['monto'].resample('ME').sum()

# 2. Rolling: Promedio móvil de 7 días (suavizar curvas)
df['promedio_7d'] = df['monto'].rolling(window=7).mean()

# 3. Shift: Comparar venta de hoy vs ayer
df['venta_ayer'] = df['monto'].shift(1)
df['variacion'] = (df['monto'] / df['venta_ayer']) - 1

```

---

### Tabla Comparativa: ¿Cuándo usar qué?

| Tarea | Herramienta | Por qué |
| --- | --- | --- |
| **Cruzar tablas** | `pd.merge()` | Relacionar dimensiones con hechos (estilo SQL). |
| **Pegar tablas** | `pd.concat()` | Unir archivos mensuales del mismo formato. |
| **Tendencias** | `.rolling()` | Para ver la dirección de los datos sin "ruido". |
| **Reportes** | `.resample()` | Para pasar de micro-datos a vistas gerenciales. |

---

> **Tip de experto:** Cuando uses `rolling()` o `resample()`, los primeros registros suelen quedar como `NaN` porque no tienen datos previos para calcular (ej. el promedio móvil del día 1 no tiene 6 días atrás). Usa `.dropna()` o `.fillna()` para manejar esos huecos antes de graficar.